# 1. Imports

Run the cell below to import the different modules used in this notebook. In order to download any that aren't already installed, please run "pip install [module name]" in a new cell

In [11]:
import mediapipe as mp
import cv2
import numpy as np
import pandas as pd
import os
import time
import torch
import torch.nn as nn

from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

import autopy
import pynput
from pynput.keyboard import Key, Controller
import webbrowser

In this notebook, OpenCV will detect video from your webcam, which MediaPipe will use to detect the 21 hand landmarks. Coordinates from these landmarks will be fed into a neural network that will output its predicted hand gesture.

<img src='https://google.github.io/mediapipe/images/mobile/hand_landmarks.png' width="900">

<font size="2">Image from https://google.github.io/mediapipe/solutions/hands.html</font>

***

Run the following two cells before moving on to part 2 or 3 of the notebook. 

The cell below has all the gestures the gestures recognition model has been trained on, as well as all the gesture recognition events in this notebook:

In [12]:
# Dictionary of all the gestures the gesture recognition model is currently trained on:
labels = {
    0: 'open palm',
    1: 'soft palm',
    2: 'palm rotated inward',
    3: 'fist',
    4: 'fist inward',
    5: 'pinch',
    6: 'open pinch',
    7: 'thumbs up',
    8: 'thumbs down',
    9: 'spock',
    10: 'heart',
    11: 'ily',
    12: 'shaka',
    13: 'metal',
    14: 'one finger up',
    15: 'two fingers up',
    16: 'three fingers up',
    17: 'four fingers up',
    18: 'up pinch one',
    19: 'up pinch two',
    20: 'up pinch three',
    21: 'up pinch four',
    22: 'mid pinch one',
    23: 'mid pinch two',
    24: 'mid pinch three',
    25: 'mid pinch four'
}

# Computer events assigned to different gestures:
palm = [0,1]
mouse_gesture = palm
stop_gesture = [3]
volume_gesture = [5,6]
end_gesture = [9]
zoom_gesture = [11]
scroll_gesture = [12]

alt_tab_gesture = [2]
pause_gesture = [10]
tab1_gesture = [14]
tab2_gesture = [15]
tab3_gesture = [16]
tablast_gesture = [17]
open_tab_gesture = [20]
close_tab_gesture = [21]
forward_tab_gesture = [19]
backward_tab_gesture = [18]
browser_forward_gesture = [18]
browser_back_gesture = [19]
link1_gesture = [22]
link2_gesture = [23]
link3_gesture = [24]
link4_gesture = [25]

The following cell's function will return flattened x, y, and z coordinates for the hand landmarks relative to the wrist's coordinates. This is how the coordinates will be fed into the model

In [13]:
def get_relative_coords(hand):
    rel_coords = []
    
    for lmk in hand.landmark:
        coords = (hand.landmark[0].x- lmk.x, 
                hand.landmark[0].y - lmk.y,
                hand.landmark[0].z - lmk.z)
        rel_coords.append(coords)
    
    rel_coords = (np.array(rel_coords)).flatten('F')
            
    return rel_coords

If you would like to train the model with your own hand gestures, please move on to part 2 of the notebook. If you would like to just run the model with the pretrained weights, please skip ahead to part 3 of the notebook.

***

# 2. Training the model

## i. Collecting coordinates for gestures from your webcam

First, initialize an empty list for all the training data and how long you would like each gesture recording to take:

In [ ]:
training_data = []
training_time = 20 # For how long the current gesture will be recorded (in seconds)

Then, set the label for what gesture you would currently like to collect coordinates for. Run the following cell each time you pick a new gesture. When I trained the model, for each gesture, I recorded one set of gestures for my right hand, and then for my right hand, and then moved on to the next gestures label

In [ ]:
curr_label = 26
print('Ready to record the following gesture: ' + labels[curr_label])

***

The following cell will run your webcam and will store the collected coords in coords_list_both. OpenCV handles the video capture. The OpenCV window will close by itself once the set training_time has passed, or earlier if you press 'q'.

In [29]:
cap = cv2.VideoCapture(0) #might have to change this if you have more than one webcam
cw, ch = 640, 480 #camera resolution
sw, sh = autopy.screen.size() #screen resolution
cap.set(3,cw)
cap.set(4,ch)
mp_drawing = mp.solutions.drawing_utils 
mp_hands = mp.solutions.hands 
coords_list_both = []
start_time = time.time()


with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5, max_num_hands=1) as hands: 
    # May need to reduce min_detection_confidence when recording certain gestures
    # Only one hand is intended to be noted down per gesture recording for training, so the maximum number of hands here is 1
    while cap.isOpened():
        ret, frame = cap.read()
    
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  
        image = cv2.flip(image, 1) 
        image.flags.writeable = False 
        results = hands.process(image) 
        image.flags.writeable = True 
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
        
        if results.multi_hand_landmarks: 
            for hand, side in zip(results.multi_hand_landmarks, results.multi_handedness): 
                # Draw all hand landmarks:
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS,
                                          mp_drawing.DrawingSpec(color=(255,255,255), thickness=1, circle_radius=5), 
                                          mp_drawing.DrawingSpec(color=(255,255,255), thickness=1), 
                                          )
                #####################################################################
                handedness = side.classification[0].label                 
                if side.classification[0].score > 0.9995: #reduces noise   
                     
                    if handedness == 'Left':      
                        coords_list = get_relative_coords(hand)
                        coords_list_both.append(coords_list) #could split up right and left hand for training

                    if handedness == 'Right':
                        coords_list = get_relative_coords(hand)
                        coords_list[:20] = -coords_list[:20] #the right hand is treated as a flipped left hand for the sake of the model    
                        coords_list_both.append(coords_list)
                
                ##################################################################### 
                          
        cv2.imshow('Gestures',image)           
        if (cv2.waitKey(10) & 0xFF == ord('q'))|((time.time() - start_time) > training_time): 
            break
                
cap.release() 
cv2.destroyAllWindows()

coords_list_both = np.insert(coords_list_both, 0, curr_label, axis=1)

Run this cell for each gesture recording or add the code to the previous cell:

In [ ]:
training_data.append(coords_list_both) 
len(training_data)

***

After collecting the coords from all of your gestures, run the following cell to put your data in a pandas dataframe:

In [ ]:
flattened_training_data =[x for sublist in training_data for x in sublist]
df = pd.DataFrame(flattened_training_data, columns = ['Label', *list(range(0, 63))])

Uncomment the following cell if you would like to append your collected coords to mine:

In [ ]:
# new_df = df
# df = (pd.read_csv('training_data.csv')).append(new_df, ignore_index=True)

Run this to see what your data looks like:

In [33]:
df

,Label,0,1,2,3,4,5,6,7,8,...,53,54,55,56,57,58,59,60,61,62
0,0.0,-0.0,-0.041926,-0.064393,-0.077695,-0.100478,-0.020028,-0.026275,-0.029684,-0.033808,...,0.094016,0.112747,0.049911,0.072864,0.094560,0.112693,0.055764,0.081397,0.099972,0.112621
1,0.0,-0.0,-0.055530,-0.092898,-0.115531,-0.141731,-0.040270,-0.063163,-0.075475,-0.085149,...,0.057199,0.078605,0.036256,0.050221,0.068569,0.084626,0.051791,0.071071,0.085861,0.096988
2,0.0,-0.0,-0.058933,-0.097290,-0.120274,-0.148960,-0.043884,-0.067256,-0.079459,-0.088954,...,0.051335,0.071981,0.032002,0.043575,0.061155,0.075661,0.046285,0.063098,0.076718,0.086555
3,0.0,-0.0,-0.062528,-0.101911,-0.125646,-0.156448,-0.044332,-0.069646,-0.083451,-0.093932,...,0.046991,0.067786,0.029085,0.039506,0.056380,0.070195,0.042327,0.058426,0.071323,0.080170
4,0.0,-0.0,-0.060706,-0.100116,-0.123723,-0.152498,-0.041669,-0.067570,-0.082614,-0.093875,...,0.064941,0.088373,0.040920,0.057121,0.078812,0.096684,0.058394,0.079690,0.095399,0.106746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25238,25.0,0.0,-0.029870,-0.044996,-0.018196,0.009353,-0.074637,-0.098222,-0.110564,-0.118064,...,0.052539,0.068158,0.015566,0.031339,0.057968,0.079122,0.013185,0.032928,0.056452,0.072856
25239,25.0,0.0,-0.029561,-0.044932,-0.020767,0.004882,-0.075192,-0.097627,-0.109392,-0.116656,...,0.060101,0.076826,0.019354,0.034823,0.062218,0.084348,0.014773,0.033055,0.055936,0.072838
25240,25.0,0.0,-0.029826,-0.044019,-0.017242,0.009939,-0.073680,-0.096601,-0.108968,-0.116440,...,0.052541,0.067963,0.014630,0.028576,0.053562,0.073606,0.010056,0.026351,0.047288,0.062207
25241,25.0,0.0,-0.030386,-0.044888,-0.017833,0.010559,-0.072931,-0.094612,-0.106458,-0.114058,...,0.046301,0.060641,0.013730,0.026451,0.051075,0.070704,0.011141,0.026840,0.047899,0.063093


Run this to save your data:

In [ ]:
filename = 'training_data_new'
df.to_csv(filename+'.csv', index=False)

***

## ii. Training a neural network for gesture recognition

Run this to prepare a custom dataset from the training data that was just collected:

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

class HandGesturesDataset(Dataset):
    def __init__(self, ):
        xy = np.loadtxt(open(filename+'.csv','rb'), delimiter=',', dtype=np.float32, skiprows=1)
        self.x = torch.from_numpy(xy[:,1:])
        self.y = (torch.from_numpy(xy[:,0])).type(torch.LongTensor)
        self.n_samples = xy.shape[0]
        
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return self.n_samples
    
dataset = HandGesturesDataset()
indices = list(range(len(dataset)))
split = int(np.floor(0.2 * len(dataset)))
np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

Run this to initialize the neural network used in this project:

In [ ]:
import pytorch_lightning as pl
import torch.nn.functional as F

input_size = 63
num_classes = len(labels)
num_epochs = 100
batch_size = 32
learning_rate = 0.001

class NeuralNet(pl.LightningModule):
    def __init__(self, input_size, num_classes):
        super(NeuralNet, self).__init__()
        self.input_size = input_size
        self.l1 = nn.Linear(input_size, 256)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(256, num_classes)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        return out
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=learning_rate)
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.log("loss", loss)        
        return loss
  
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.log('val_loss', loss) 
           
    def train_dataloader(self):
        train_dataloader = DataLoader(dataset=dataset, batch_size=batch_size, sampler=train_sampler)
        return train_dataloader

    def val_dataloader(self):
        val_dataloader = DataLoader(dataset=dataset, batch_size=batch_size, sampler=val_sampler)
        return val_dataloader
    

Run this to train the neural network:

In [ ]:
from pytorch_lightning import Trainer

trainer = Trainer(auto_lr_find=True, max_epochs=num_epochs, gpus=1, fast_dev_run=False)
model = NeuralNet(input_size, num_classes)
trainer.fit(model)

Uncomment this if you would like to run TensorBoard:

In [ ]:
# # %load_ext tensorboard
# %tensorboard --logdir ./lightning_logs

Uncomment this if you would like to save your model's weights:

In [ ]:
#torch.save(model.state_dict(), 'gestures_model.pth')

***

# 3. Running the model for gesture recognition and gesture recognition events

Initialize the neural network if you haven't done so already in part 2:

In [15]:
import pytorch_lightning as pl
import torch.nn.functional as F

input_size = 63
num_classes = len(labels)

class NeuralNet(pl.LightningModule):
    def __init__(self, input_size, num_classes):
        super(NeuralNet, self).__init__()
        self.input_size = input_size
        self.l1 = nn.Linear(input_size, 256)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(256, num_classes)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        return out

Run the following cell if you would like to use the pretrained weights:

In [21]:
model = NeuralNet(input_size, num_classes)
model.load_state_dict(torch.load('100epochs.pth')) 

<All keys matched successfully>

Run the following cell to define some general helper functions:

In [22]:
# Feeds hand coordinates through the trained model to get its predicted hand gesture
def get_gesture(coords_list):
    model_out = model(torch.tensor(coords_list.astype(np.float32)))
    gesture = torch.argmax(model_out).item()
    return gesture


# Returns a distance in 2D or in 3D between two points
def get_distance(a, b, hand, xyz=True):
    distance = 0
    if xyz == True:
        distance = np.cbrt((hand.landmark[a].x - hand.landmark[b].x)**2 
                           + (hand.landmark[a].y - hand.landmark[b].y)**2
                           + (hand.landmark[a].z - hand.landmark[b].z)**2)
    else:
        distance = np.sqrt((hand.landmark[a].x - hand.landmark[b].x)**2 
                           + (hand.landmark[a].y - hand.landmark[b].y)**2)
    return distance

# Returns gestures if they match ones recorded in previous time steps (to to be able to modulate how responsive the different gesture events should be)
def get_gesture_confirmation(gestures):
    short, med, long = None, None, None
    if gestures[-1] == gestures[-2]:
        short = gestures[-1]
        if gestures[-1] == gestures[-2] == gestures[-3]:
            med = gestures[-1]
            if gestures[-1] == gestures[-2] == gestures[-3] == gestures[-4]:
                long = gestures[-1]
    return short, med, long

Run the following cell to define functions for running different hand gesture events:

In [26]:
# The code for the hand mouse is in part based on the code from this tutorial: https://www.youtube.com/watch?v=8gPONnGIPgw
def hand_mouse(hand, image, prev_x, prev_y):
    smoothing = 3
    bounds_x = cw/3
    bounds_y = ch/3
    
    x, y = hand.landmark[9].x, hand.landmark[9].y #the mouse coords here are obtained from the middle finger MCP to keep the pointer stable when clicking
    
    x = np.interp(x*cw, (bounds_x,cw-bounds_x),(0,sw))
    y = np.interp(y*ch, (bounds_y,ch-bounds_y),(0,sh))

    curr_x = prev_x + (x - prev_x)/smoothing
    curr_y = prev_y + (y - prev_y)/smoothing
    
    autopy.mouse.move(curr_x, curr_y)
    
    # Distance between the tips of the index and middle fingers divided by index finger length to control for distance away from the camera
    click_distance = (get_distance(8, 12, hand,xyz=False) / get_distance(5, 8, hand, xyz=False)) 
    if click_distance < 0.2: 
        autopy.mouse.click()

    return curr_x, curr_y
####################################################################################################
# The code for changing volume is based on the code from this tutorial: https://www.youtube.com/watch?v=9iEPzbG-xLE
def hand_change_volume(hand): 
    devices = AudioUtilities.GetSpeakers()
    interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
    volume = cast(interface, POINTER(IAudioEndpointVolume))
    smoothing = 5
    
    x4, y4 = int((hand.landmark[4].x)*cw), int((hand.landmark[4].y)*ch)
    x8, y8 = int((hand.landmark[8].x)*cw), int((hand.landmark[8].y)*ch)
    
    cv2.line(image, (x4, y4), (x8, y8),(255,255,255),3)
    length = (get_distance(4, 8, hand,xyz=False) / get_distance(5, 8, hand, xyz=False))*100                     
    vol_bar = np.interp(length, [17,175],[400, 150])
    vol_per = np.interp(length,[17,175],[0,100])
    vol_per = smoothing * round(vol_per/smoothing)
    
    volume.SetMasterVolumeLevelScalar(vol_per/100, None)

    cv2.rectangle(image, (40, 150), (80, 425),(255,255,255), 3)
    cv2.rectangle(image, (40, int(vol_bar)), (80, 425),(255,255,255), cv2.FILLED)
    curr_vol = int(volume.GetMasterVolumeLevelScalar()*100)
    cv2.putText(image,f'{int(curr_vol)}%',(40,140), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255),3)
####################################################################################################    
    
def hand_scroll(hand):
    smoothing = 700
    bounds_x = cw/3
    bounds_y = ch/3
    
    x, y = hand.landmark[9].x, hand.landmark[9].y
    
    x = np.interp(x*cw, (bounds_x,cw-bounds_x),(0,sw))
    y = np.interp(y*ch, (bounds_y,ch-bounds_y),(0,sh))

    x_rel_center = (sw/2 - x)/smoothing
    y_rel_center = (sh/2 - y)/smoothing
    
    mouse.scroll(x_rel_center, y_rel_center)
####################################################################################################    
    
def hand_zoom(hand, prev_y):
    y = hand.landmark[9].y
    
    if y < prev_y:
        with keyboard.pressed(Key.ctrl):
            keyboard.press('=')
            keyboard.release('=')  

    if y > prev_y:
        with keyboard.pressed(Key.ctrl):
            keyboard.press('-')
            keyboard.release('-')  
            
    next_zoom = time.time() + 1

    return y, next_zoom

####################################################################################################
def hand_nonmode_event(lgesture, rgesture):
    event_text = ''
    gesture = None
    
    # Can specifiy if the gesture should only be read from the left or right hand, or either hand
    if rgesture:
        gesture = rgesture    
    if lgesture:
        gesture = rgesture # If both hands are up, this prioritizes the left hand for either-hand gestures
        
        
        
    # Pause (spacebar):
    if gesture in alt_tab_gesture:
        with keyboard.pressed(Key.alt):
            keyboard.press(Key.tab)
            keyboard.release(Key.tab) 
        event_text = 'alt tab'
        
        
    # Alt-tab        
    if gesture in pause_gesture:
        keyboard.press(Key.space)
        keyboard.release(Key.space)   
        pause_mode = False
        pause_delay = time.time()  
        event_text = 'pause'
        
        
    # Cycling through tabs, opening or closing tabs:
    if rgesture in tab1_gesture:
        with keyboard.pressed(Key.ctrl):
            keyboard.press('1')
            keyboard.release('1')  
        event_text = 'go to tab 1'
    if rgesture in tab2_gesture:
        with keyboard.pressed(Key.ctrl):
            keyboard.press('2')
            keyboard.release('2') 
        event_text = 'go to tab 2'
    if rgesture in tab3_gesture:
        with keyboard.pressed(Key.ctrl):
            keyboard.press('3')
            keyboard.release('3') 
        event_text = 'go to tab 3'
    if rgesture in tablast_gesture:
        with keyboard.pressed(Key.ctrl):
            keyboard.press('9')
            keyboard.release('9')  
        event_text = 'go to the last tab'
        
    if rgesture in backward_tab_gesture:
        with keyboard.pressed(Key.ctrl):
            with keyboard.pressed(Key.shift):
                keyboard.press(Key.tab)
        keyboard.release(Key.tab)      
        event_text = 'cycle backward through tabs'    
    if rgesture in forward_tab_gesture:
        with keyboard.pressed(Key.ctrl):
            keyboard.press(Key.tab)
        keyboard.release(Key.tab)  
        event_text = 'cycle forward through tabs'        
        
    if rgesture in open_tab_gesture:  # I've found that this one doesn't get captured too efficiently
        with keyboard.pressed(Key.ctrl):
            keyboard.press('t')
            keyboard.release('t') 
        event_text = 'open new tab'
            
    if rgesture in close_tab_gesture: # found this one to be finicky, too 
        with keyboard.pressed(Key.ctrl):
            keyboard.press('w')
            keyboard.release('w') 
        event_text = 'close tab or window'
                
            
    # Going forward or back one page in a browser:
    if lgesture in browser_forward_gesture:
        with keyboard.pressed(Key.alt):
            keyboard.press(Key.right)
            keyboard.release(Key.right) 
        event_text = 'go forward'
            
    if lgesture in browser_back_gesture:
        with keyboard.pressed(Key.alt):
            keyboard.press(Key.left)
            keyboard.release(Key.left) 
        event_text = 'go back'
        
            
    # Links:
    if rgesture in link1_gesture:
        webbrowser.open('https://google.github.io/mediapipe/solutions/hands.html')
        event_text = 'open link'
    if rgesture in link2_gesture:
        webbrowser.open('https://opencv.org/')  
        event_text = 'open link'
    if rgesture in link3_gesture:
        os.startfile('example file.txt') 
        event_text = 'open file'
    if rgesture in link4_gesture:
        webbrowser.open('https://www.youtube.com/watch?v=dQw4w9WgXcQ')
        event_text = 'open link'
            
    return event_text

***

The following Python cell will turn on your webcam and will run the gesture recognition events. If you would like to close the OpenCV window that opens up, press "q" or do the end gesture in front of your webcam, which is currently set as a Vulcan salute (see below)

<img src='https://api.time.com/wp-content/uploads/2015/04/464967684.jpg?w=800&quality=85' width="500">

In [28]:
cap = cv2.VideoCapture(0)
cw, ch = 640, 480 #camera resolution
sw, sh = autopy.screen.size() #screen resolution
cap.set(3,cw)
cap.set(4,ch)
keyboard = Controller()
mouse = pynput.mouse.Controller()
mp_drawing = mp.solutions.drawing_utils 
mp_hands = mp.solutions.hands 

#############################################################################################
mouse_mode, volume_mode, end_mode, scroll_mode, zoom_mode = False, False, False, False, False 

recorded_time, event_time, zoom_time = 0, 0, 0
left_gestures, right_gestures = [], []
event_text = ''
get_time = True
skipped = False
time_delay = 0.25  #this delay can be increased if you would like gesture sensivity to increase, and vice versa
prev_x, prev_y = sw/2, sh/2


nones = lambda n: [None for _ in range(n)]
[cl_s, 
 cr_s, 
 cl_m, 
 cr_m, 
 cl_l, 
 cr_l,
 lgesture,
 rgesture] = nones(8)

#############################################################################################

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5, max_num_hands=1) as hands: #detection: threshold for initial detection, tracking: threshold for tracking after detection, default max num of hands = 2
    while cap.isOpened():
        ret, frame = cap.read()
    
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)   
        image = cv2.flip(image, 1) 
        image.flags.writeable = False 
        results = hands.process(image) 
        image.flags.writeable = True 
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  

        
        if results.multi_hand_landmarks: 
            for hand, side in zip(results.multi_hand_landmarks, results.multi_handedness): 
                # Draws the landmarks:
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS,
                                          mp_drawing.DrawingSpec(color=(255,255,255), thickness=1, circle_radius=5), 
                                          mp_drawing.DrawingSpec(color=(255,255,255), thickness=1),
                                          )
                #####################################################################
                # Figures out which hand's up and sets off some gesture events
                handedness = side.classification[0].label                 
                if side.classification[0].score > 0.9995: #reduces noise   
                    skipped = False
                     
                    if handedness == 'Left':      
                        coords_list = get_relative_coords(hand)
                        lgesture = get_gesture(coords_list)
                        
                        if volume_mode == True:
                            volume_mode = False
                    if handedness == 'Right':
                        coords_list = get_relative_coords(hand)
                        coords_list[:20] = -coords_list[:20]
                        rgesture = get_gesture(coords_list) 
                    
                        if mouse_mode == True:   
                            prev_x, prev_y = hand_mouse(hand, image, prev_x, prev_y)
                            event_time = time.time() + 1
                            event_text = 'mouse mode'
                        if scroll_mode == True:
                            hand_scroll(hand)
                            event_time = time.time() + 1
                        if (zoom_mode == True)& ((time.time() - zoom_time) > 0):
                            prev_y, zoom_time = hand_zoom(hand, prev_y)
                            event_time = time.time()+ 1
                            event_text = 'zoom mode'
                        if volume_mode == True: 
                            hand_change_volume(hand)
                            event_time = time.time() + 1
                            event_text = 'volume mode'
                else:
                    skipped = True
                
                ##################################################################### 
            
        ###################################################################################### 
        # This section records a hand gesture for each hand every time delay. Gestures consistent across multiple time steps are returned
        if skipped == False:
            if get_time == True:
                recorded_time = time.time()
                get_time = False
            if (get_time == False) & ((time.time()- recorded_time) > time_delay):
                get_time = True          

                if 'Left' in str(results.multi_handedness): #if left hand is on screen
                    left_gestures.append(lgesture)
                else:
                    left_gestures = []
                    lgesture, cl_s, cl_m, cl_l = None, None, None, None

                if 'Right' in str(results.multi_handedness): #if right hand is on screen
                    right_gestures.append(rgesture)
                else:
                    right_gestures = []
                    rgesture, cr_s, cr_m, cr_l = None, None, None, None


            if len(left_gestures) > 3:
                cl_s, cl_m, cl_l = get_gesture_confirmation(left_gestures) #use cl_l v.s. cl_s, for example, if you would like an event to be initiated after a gesture has been up (and captured) for a longer period of time
                

            if len(right_gestures) > 3:
                cr_s, cr_m, cr_l = get_gesture_confirmation(right_gestures)

            ######################################################################################## 

            #############################################################################################################
            # This section handles the triggers for different events
            if (cr_s in mouse_gesture) & (mouse_mode == False) & (cl_m == None):
                mouse_mode = True
                event_text = 'mouse mode'
            if (mouse_mode == True) & ((not cr_s in mouse_gesture) | (not cl_m == None)):
                mouse_mode = False
                event_text = ''
                
            if (cr_m in volume_gesture) & (volume_mode == False) & (cl_m == None) & ((time.time() - event_time) > 1):
                volume_mode = True
                event_text = 'volume mode'
            if (volume_mode == True)&((not cr_s in volume_gesture) | (not cl_m == None)):
                volume_mode = False
                event_text = ''

            if (cr_m in zoom_gesture) & (zoom_mode == False):
                zoom_mode = True
                event_text = 'zoom mode'
            if (zoom_mode == True) & (not cr_s in zoom_gesture):
                zoom_mode = False
                event_text = ''
 
            if (cr_m in scroll_gesture) & (scroll_mode == False):
                scroll_mode = True
                event_text = 'scroll mode'
            if (scroll_mode == True) & (not cr_s in scroll_gesture):
                scroll_mode = False
                event_text = ''      
    
    

            if ((not cl_l == None) | (not cr_l == None)):
                if (time.time() - event_time) > 1.5:
                    event_text = hand_nonmode_event(cl_l, cr_l)
                    event_time = time.time()
                    
            if (cr_l in end_gesture):
                end_mode = True
                event_text = 'bye'   

                    
            ###############################################################################################################
            # This section handles the text that shows up on the screen
            if ((not lgesture == None) & (not rgesture == None)):
                cv2.putText(image, 'Left hand: ' + labels[lgesture], [25,45], cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                cv2.putText(image, 'Right hand: ' + labels[rgesture], [25,85], cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            else:
                if not lgesture == None:
                    cv2.putText(image, labels[lgesture], [25,45], cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                if not rgesture == None:
                    cv2.putText(image, labels[rgesture], [25,45], cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            if event_text:
                cv2.putText(image, event_text, [25,ch-10], cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        if (time.time() - event_time) > 2:
            event_text = ''
                
        cv2.imshow('Gestures',image)     
        
        if (cv2.waitKey(10) & 0xFF == ord('q')) | (end_mode == True): 
            break              
                
cap.release() 
cv2.destroyAllWindows()